In [1]:
from datastructures.workflow import SOLVER, COMP, ENDCOMP, path, order_from_tree
from datastructures.graphutils import end_components, solver_children
from collections import OrderedDict, defaultdict

# Mock order from tree

In [2]:
Ftree = OrderedDict([(0,0), (1,1),(2,2),(3,2),(4,1,),(5,1),(6,1),(7,0),(8,0)])
Stree = {2:1, 1:0}
endcomponents = [0,2,5,7]
Eout = {comp: (None,) if comp in [0,2,5,7] else (99,) for comp,_ in Ftree.items()}

### Test method from library

In [3]:
order_from_tree(Ftree, Stree, Eout, mergeendcomp=False)

[(SOLVER, 0, None),
 (SOLVER, 1, 0),
 (COMP, 1, 1),
 (SOLVER, 2, 1),
 (COMP, 3, 2),
 (ENDCOMP, 2, 2),
 (COMP, 4, 1),
 (COMP, 6, 1),
 (ENDCOMP, 5, 1),
 (COMP, 8, 0),
 (ENDCOMP, 0, 0),
 (ENDCOMP, 7, 0)]

### Debug method

In [4]:
includesolver = True
mergeendcomp = False

In [5]:
endcomps = {key: None in var for key,var in Eout.items()}
visited_solvers = set()
sequence = []
endcompqueue = defaultdict(list)
queue = list(Ftree.items())
while queue:
    component, parent = queue.pop(0)
    ancestors = path(Stree, parent, visited_solvers)
    reverse_ancestors = ancestors[::-1]
    visited_solvers = visited_solvers.union(reverse_ancestors)
    if includesolver:
        sequence += [(SOLVER, solver, Stree.get(solver,None)) for solver in reverse_ancestors]
    if endcomps[component]:
        endcompqueue[parent].append(component)
    else:
        sequence += [(COMP, component, parent)]
    remainingcomps = len([elt for elt in solver_children(dict(queue), parent)])
    lastchildcomp = remainingcomps==0
    if lastchildcomp:
        if mergeendcomp and endcompqueue.get(parent,False):
            sequence += [(ENDCOMP, endcompqueue[parent], parent)]
        else:
            sequence += [(ENDCOMP, endcomp, parent) for endcomp in endcompqueue[parent]]

In [6]:
sequence

[(SOLVER, 0, None),
 (SOLVER, 1, 0),
 (COMP, 1, 1),
 (SOLVER, 2, 1),
 (COMP, 3, 2),
 (ENDCOMP, 2, 2),
 (COMP, 4, 1),
 (COMP, 6, 1),
 (ENDCOMP, 5, 1),
 (COMP, 8, 0),
 (ENDCOMP, 0, 0),
 (ENDCOMP, 7, 0)]